In [1]:
import pandas as pd
import esig.tosig as ts
import numpy as np

In [2]:
file_path = "./datacsv/processed_battery_discharge_data_filtered.xlsx"
xls = pd.ExcelFile(file_path, engine='openpyxl')

In [3]:
# Set the signature truncation depth
signature_level = 3
result_data = []

for sheet_name in xls.sheet_names:
    df = xls.parse(sheet_name)
    if 'cycle' in df.columns and 'time' in df.columns and 'voltage' in df.columns and 'capacity' in df.columns:
        for cycle, group in df.groupby('cycle'):
            path = group[['time', 'voltage']].to_numpy()
            if len(path) < 2:
                continue
            try:
                sig = ts.stream2sig(path, signature_level)
            except:
                continue
            cap = group['capacity'].iloc[0]
            last_time = group['time'].iloc[-1] 
            last_voltage = group['voltage'].iloc[-1]

            label = last_voltage == 0  

            record = {
                'sheet_cycle': f'{sheet_name}_{cycle}',
                'capacity': cap,
                'event': int(label), 
                'time': last_time
            }
            for idx, val in enumerate(sig):
                record[f's_{idx}'] = val
            result_data.append(record)

df_result = pd.DataFrame(result_data)
df_result.to_csv("./datacsv/battery_signature_features.csv", index=False)

In [4]:
# Corrected file path
file_path = './datacsv/battery_signature_features.csv'  # Make sure this path is correct

# Load the CSV file
df = pd.read_csv(file_path)

# Show the first 5 rows
print("First 5 rows of the CSV file:")
print(df.head())

First 5 rows of the CSV file:
         sheet_cycle  capacity  event      time  s_0       s_1    s_2  \
0  B0005_discharge_1  1.856487      0  2513.187  1.0  2496.406 -1.675   
1  B0005_discharge_2  1.846327      0  2514.438  1.0  2497.766 -1.676   
2  B0005_discharge_3  1.835349      0  2514.734  1.0  2497.937 -1.679   
3  B0005_discharge_4  1.835263      0  2515.203  1.0  2498.437 -1.681   
4  B0005_discharge_5  1.834646      0  2513.813  1.0  2497.094 -1.680   

            s_3         s_4          s_5       s_6           s_7  \
0  3.116021e+06 -482.601060 -3698.878990  1.402813  2.592952e+09   
1  3.119417e+06 -470.485739 -3715.770077  1.404488  2.597192e+09   
2  3.119845e+06 -473.417370 -3720.618853  1.409521  2.597725e+09   
3  3.121094e+06 -475.651763 -3724.220834  1.412881  2.599285e+09   
4  3.117739e+06 -475.597514 -3719.520406  1.411200  2.595096e+09   

             s_8            s_9       s_10          s_11        s_12  \
0 -350144.612167 -504478.957456  77.224488 -4.3647